In [1]:
import os
from torch.utils.data import DataLoader,Dataset
from tqdm import tqdm
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import pandas as pd
from torchvision import models
import torch.nn as nn
import pytesseract
# from fastai.vision import Path
import torch
from torch.autograd import Variable
import cv2
from imutils import contours

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
# ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
ALL_CHAR_SET = NUMBER
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 5

def encode(a):
    onehot = [0]*ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] += 1
    return onehot

In [3]:
class Mydataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx].img
        img = Image.open(img_path)
        img = img.convert('L')
        label = self.df.iloc[idx].label
        # label = Path(self.path/img_path).name[:-4]
        label_oh = []
        for i in label:
            label_oh += encode(i)
        if self.transform is not None:
            img = self.transform(img)
        return img, np.array(label_oh), label
    
    def __len__(self):
        return self.df.shape[0]



In [4]:
from glob import glob
import os
all_files = glob(os.path.join("..", "captcha", "fix", "captcha-*.png"))
df = pd.DataFrame({'img': all_files})
df["label"] = df.img.apply(lambda x: os.path.basename(x).split("-")[1].split(".")[0].split("-")[0])
df.head()
train_df, test_df = df.iloc[:int(df.shape[0]*0.8)], df.iloc[int(df.shape[0]*0.8):]
print(train_df.shape, test_df.shape)

(3753, 2) (939, 2)


In [36]:
from multiprocessing import cpu_count, Process, Lock, Semaphore, Pool

box_df = pd.DataFrame(columns=[
                      "img", "label", "label", "index", "x1", "x2", "y1", "y2"])

lock = Lock()
semaphore = Semaphore(cpu_count()-1)
# thread_progress = tqdm(total=df.shape[0])


def get_to_box(row):

    series_list = []
    original_img = cv2.imread(row.img)  # Load the upsampled image
    img = cv2.cvtColor(original_img, cv2.COLOR_BGR2HSV)
    msk = cv2.inRange(img, np.array([50, 50, 50]), np.array([179, 255, 255]))
    krn = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 3))
    dlt = cv2.dilate(msk, krn, iterations=1)
    thr = 255 - cv2.bitwise_and(dlt, msk)

    # txt = pytesseract.image_to_string(
    #     thr, config='-c  tessedit_char_whitelist=0123456789 --psm 6')
    # print([t for t in txt if t.isalnum()])
    boxes = pytesseract.image_to_boxes(
        thr, config='-c  tessedit_char_whitelist=0123456789 --psm 6')
    # img = thr.copy()

    txt = "".join([t[0] for t in boxes.splitlines()])
    print("txt", txt)
    if len(txt) == len(row.label):

        lock.acquire()
        for index, box in enumerate(boxes.splitlines()):
            box = box.split()
            series = pd.Series({
                "img": row.img,
                "box_label": row.label[index],
                "label": row.label,
                "index": index,
                "x1": img.shape[0] - int(box[4]),
                "x2": img.shape[0] - int(box[2]),
                "y1": int(box[1]),
                "y2": int(box[3]),
            })
            series_list.append(series)
        lock.release()
    return series_list
    # thread_progress.update(1)
    # semaphore.release()


In [37]:
i = 0
all_processes = []
# for index, row in tqdm(df.iterrows(), total=df.shape[0]):
#     p = Process(target=get_to_box, args=(row,))
#     all_processes.append(p)
#     semaphore.acquire()
#     p.start()

# for p in all_processes:
#     p.join()
s_list = []
from concurrent.futures import ProcessPoolExecutor, wait, as_completed
# for index, row in tqdm(df.iterrows():
with ProcessPoolExecutor(max_workers=cpu_count()-1) as executor:
    with tqdm(total=df.shape[0]) as p_bar:
        futures_list = []
        for index, row in df.iterrows():
            future = executor.submit(get_to_box, row)
            future.add_done_callback(lambda p: p_bar.update())
            futures_list.append(future)
        
        for future in futures_list:
            result = future.result()
            if result:
                s_list = s_list + result



b = pd.DataFrame(s_list)

  0%|          | 0/4692 [00:00<?, ?it/s]

txttxt  
txt
 

  0%|          | 1/4692 [00:01<1:29:16,  1.14s/it]


txt 
txt 42354


  0%|          | 5/4692 [00:01<18:30,  4.22it/s]  


In [39]:
b

,img,box_label,x1,x2,y1,y2
0,../captcha/fix/captcha-42851.png,4,10,42,13,33
1,../captcha/fix/captcha-42851.png,2,11,40,33,54
2,../captcha/fix/captcha-42851.png,8,10,42,45,75
3,../captcha/fix/captcha-42851.png,5,11,41,57,91
4,../captcha/fix/captcha-42851.png,1,12,40,91,107


In [25]:
b.head()

""


In [ ]:
b.write_csv("box_df.csv")

In [7]:
transform = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
])
train_ds = Mydataset(train_df, transform=transform)
test_ds = Mydataset(test_df, transform=transform)
train_dl = DataLoader(train_ds, batch_size=32, num_workers=10, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32, num_workers=10, shuffle=True)


In [8]:
model = models.resnet18(pretrained=False)

/home/amir/.pyenv/versions/3.11.1/envs/automate_news/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/amir/.pyenv/versions/3.11.1/envs/automate_news/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN*MAX_CAPTCHA, bias=True)

In [10]:
model = model.to(device)

In [11]:
loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
for epoch in range(20):
    for step, i in enumerate(train_dl):
        img, label_oh, label = i
        img = Variable(img).cuda()
        label_oh = Variable(label_oh.float()).cuda()
        pred = model(img)
        loss = loss_func(pred, label_oh)
        optm.zero_grad()
        loss.backward()
        optm.step()
        print('eopch:', epoch+1, 'step:', step+1, 'loss:', loss.item())

eopch: 1 step: 1 loss: 0.7424418330192566
eopch: 1 step: 2 loss: 0.5587933659553528
eopch: 1 step: 3 loss: 0.4174691438674927
eopch: 1 step: 4 loss: 0.35221850872039795
eopch: 1 step: 5 loss: 0.3326190710067749
eopch: 1 step: 6 loss: 0.3254120349884033
eopch: 1 step: 7 loss: 0.3261052966117859
eopch: 1 step: 8 loss: 0.331088125705719
eopch: 1 step: 9 loss: 0.3397544324398041
eopch: 1 step: 10 loss: 0.3322392702102661
eopch: 1 step: 11 loss: 0.3339664041996002
eopch: 1 step: 12 loss: 0.32724979519844055
eopch: 1 step: 13 loss: 0.32407498359680176
eopch: 1 step: 14 loss: 0.327959269285202
eopch: 1 step: 15 loss: 0.3198370337486267
eopch: 1 step: 16 loss: 0.3163624703884125
eopch: 1 step: 17 loss: 0.31956231594085693
eopch: 1 step: 18 loss: 0.3294880986213684
eopch: 1 step: 19 loss: 0.31840401887893677
eopch: 1 step: 20 loss: 0.31586265563964844
eopch: 1 step: 21 loss: 0.32094481587409973
eopch: 1 step: 22 loss: 0.3189174234867096
eopch: 1 step: 23 loss: 0.3077778220176697
eopch: 1 step: 

KeyboardInterrupt: 

In [ ]:
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
correct, wrong = 0, 0
for step, (img, label_oh, label) in enumerate(test_dl):
    img = Variable(img).cuda()
    preds = model(img)
    for pred in preds:
        # pred = preds[0]
        c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
        c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
        c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
        c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
        c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
        c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)
        if c == label[0]:
            correct += 1
        else:
            wrong += 1
            # print('label:', label[0], 'pred:', c)


print("correct", correct, "wrong", wrong, "acc", correct/(correct+wrong))


correct 24 wrong 784 acc 0.0297029702970297
